# Handling nultiple inputs

When you want to send a subset of the features through the wide path and a different subset(possibly overlapping) through the deep path? One solution is to use multiple inputs.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

X_train.shape[1:]

import tensorflow as tf
from tensorflow import keras

In [2]:
input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_A, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [11]:
model.compile(loss='mse', optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [12]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [13]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20, validation_data=((X_valid_A, X_valid_B), y_valid))

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 2.2209 - val_loss: 1.0822
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.9692 - val_loss: 0.8441
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.8197 - val_loss: 0.7675
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.7532 - val_loss: 0.7199
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7081 - val_loss: 0.6828
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6732 - val_loss: 0.6537
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6455 - val_loss: 0.6301
Epoch 8/20
363/363 [==============================] - 2s 5ms/step - loss: 0.6227 - val_loss: 0.6098
Epoch 9/20
363/363 [==============================] - 1s 4ms/step - loss: 0.6038 - val_loss: 0.5931
Epoch 10/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5875 - val_loss: 0.5797

In [15]:
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
mse_test

162/162 [==============================] - 0s 2ms/step - loss: 0.5084


0.5084089636802673

In [17]:
y_pred = model.predict((X_new_A, X_new_B))
y_pred

1/1 [==============================] - 0s 54ms/step


array([[2.0000854],
       [2.150784 ],
       [2.1399703]], dtype=float32)